In [1]:
import aqt

from anki import scheduler_pb2

from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import scipy

import datetime, itertools, time

In [2]:
col = mw.col

In [6]:
restore_indices_commands = [
    "CREATE INDEX ix_notes_usn on notes (usn);",
    "CREATE INDEX ix_cards_usn on cards (usn);",
    "CREATE INDEX ix_cards_nid on cards (nid);",
    "CREATE INDEX ix_cards_sched on cards (did, queue, due);",
    "CREATE INDEX ix_notes_csum on notes (csum);",
    "CREATE UNIQUE INDEX idx_fields_name_ntid ON fields (name, ntid);",
    "CREATE UNIQUE INDEX idx_templates_name_ntid ON templates (name, ntid);",
    "CREATE INDEX idx_templates_usn ON templates (usn);",
    "CREATE UNIQUE INDEX idx_notetypes_name ON notetypes (name);",
    "CREATE INDEX idx_notetypes_usn ON notetypes (usn);",
    "CREATE UNIQUE INDEX idx_decks_name ON decks (name);",
    "CREATE INDEX idx_notes_mid ON notes (mid);",
    "CREATE INDEX idx_cards_odid ON cards (odid) WHERE odid != 0;",
    "CREATE INDEX idx_graves_pending ON graves (usn);",
    "CREATE INDEX ix_revlog_cid on revlog (cid);",
]

In [7]:
for command in restore_indices_commands:
    try:
        col.db.execute(command)
    except Exception as e:
        print(f"Rolling back after exception: {e}")
        col.db.rollback()
    finally:
        col.db.commit()

Rolling back after exception: DbError { info: "SqliteFailure(Error { code: Unknown, extended_code: 1 }, Some(\"index ix_notes_usn already exists\"))", kind: Other }
Rolling back after exception: DbError { info: "SqliteFailure(Error { code: Unknown, extended_code: 1 }, Some(\"index ix_cards_usn already exists\"))", kind: Other }
Rolling back after exception: DbError { info: "SqliteFailure(Error { code: Unknown, extended_code: 1 }, Some(\"index ix_cards_nid already exists\"))", kind: Other }
Rolling back after exception: DbError { info: "SqliteFailure(Error { code: Unknown, extended_code: 1 }, Some(\"index ix_cards_sched already exists\"))", kind: Other }
Rolling back after exception: DbError { info: "SqliteFailure(Error { code: Unknown, extended_code: 1 }, Some(\"index ix_notes_csum already exists\"))", kind: Other }
Rolling back after exception: DbError { info: "SqliteFailure(Error { code: Unknown, extended_code: 1 }, Some(\"index idx_fields_name_ntid already exists\"))", kind: Other }

In [9]:
print(list(col.db.execute('''select * from reviews where id = 1673309520000''')))
print(list(col.db.execute('''select * from revlog where id = 1673309520000''')))
print(list(col.db.execute('''select * from review_notes where id = 1673309520000''')))

[]
[[1673309520000, 1659061121013, 2015, 4, 2, -600, 2850, 5500, 2]]
[[1673309520000, '', '', 0, '']]


In [10]:
print(list(col.db.execute('''select last_insert_rowid()''')))

[[0]]
1673640296.995 GET /_anki/legacyPageData
1673640296.997 GET /_anki/legacyPageData
1673640297.006 GET /_anki/css/webview.css
1673640297.008 GET /_anki/css/toolbar.css
1673640297.009 GET /_anki/css/toolbar-bottom.css
1673640297.009 GET /_anki/js/webview.js
1673640297.010 GET /_anki/css/webview.css
1673640297.010 GET /_anki/js/vendor/jquery.min.js
1673640297.015 GET /_anki/css/deckbrowser.css
1673640297.016 GET /_anki/js/deckbrowser.js
1673640297.016 GET /_anki/js/vendor/jquery.min.js
1673640297.016 GET /_anki/js/vendor/jquery-ui.min.js
1673640297.017 GET /_anki/js/webview.js
1673640297.018 GET /_anki/imgs/gears.svg


In [16]:
print(list(col.db.execute('''INSERT INTO reviews (id, cid, usn, ease, ivl, lastIvl, factor, time, type, feedback, tags, mod) VALUES (9999999999999, 0, -1, 0, 0, 0, 0, 0, 0, '', '', 0)''')))
print(list(col.db.execute('''SELECT last_insert_rowid()''')))

[]
[[0]]


In [20]:
print(list(col.db.execute('''
CREATE TABLE IF NOT EXISTS last_inserted_row (
  table_name text,
  id integer
);
''')))


print(list(col.db.execute('''
DROP TRIGGER IF EXISTS tr_aft_ins_on_revlog;
''')))

print(list(col.db.execute('''
CREATE TRIGGER IF NOT EXISTS tr_aft_ins_on_revlog
  AFTER INSERT ON revlog
  BEGIN
    UPDATE last_inserted_row SET id = 0 WHERE table_name = 'revlog';
    UPDATE last_inserted_row SET id = 0 WHERE table_name = 'review_notes';
    INSERT INTO review_notes VALUES (NEW.id, "", "", 0, 'D');
    UPDATE last_inserted_row SET id = NEW.id WHERE table_name = 'revlog';
    UPDATE last_inserted_row SET id = NEW.id WHERE table_name = 'review_notes';
  END;
''')))

print(list(col.db.execute('''
DROP TRIGGER IF EXISTS tr_nstd_ins_on_reviews;
''')))

print(list(col.db.execute('''
CREATE TRIGGER IF NOT EXISTS tr_nstd_ins_on_reviews
  INSTEAD OF INSERT ON reviews
  BEGIN
    UPDATE last_inserted_row SET id = NEW.id WHERE table_name = 'reviews';
    
    INSERT INTO revlog
      VALUES (NEW.id, NEW.cid, NEW.usn, NEW.ease, NEW.ivl, NEW.lastIvl, new.factor, NEW.time, NEW.type)
    ON CONFLICT (id) DO
      UPDATE SET (cid, usn, ease, ivl, lastIvl, factor, time, type) = (NEW.cid, NEW.usn, NEW.ease, NEW.ivl, NEW.lastIvl, NEW.factor, NEW.time, NEW.type);

    INSERT INTO review_notes
      VALUES (NEW.id, NEW.feedback, NEW.tags, NEW.mod, 'D')
    ON CONFLICT (id) DO
      UPDATE SET (feedback, tags, mod, vis) = (NEW.feedback, NEW.tags, NEW.mod, (
            SELECT (CASE WHEN review_notes.vis == 'D' THEN 'D' ELSE 'V' END) FROM review_notes WHERE review_notes.id == NEW.id)
    );

    UPDATE last_inserted_row SET id = NEW.id WHERE table_name = 'reviews';
  END;
''')))

col.db.execute('COMMIT')

[]
[]
[]
[]
[]


[]

In [21]:
print(list(col.db.execute('''DELETE FROM reviews WHERE id = 9999999999999;''')))
print(list(col.db.execute('''INSERT INTO reviews (id, cid, usn, ease, ivl, lastIvl, factor, time, type, feedback, tags, mod) VALUES (9999999999999, 0, -1, 0, 0, 0, 0, 0, 0, '', '', 0);''')))
print(list(col.db.execute('''SELECT * FROM last_inserted_row;''')))

[]
[]
[['revlog', 9999999999999], ['review_notes', 9999999999999], ['reviews', 9999999999999]]


In [18]:
print(list(col.db.execute('''INSERT INTO last_inserted_row (table_name, id) VALUES ('revlog', 0);''')))
print(list(col.db.execute('''INSERT INTO last_inserted_row (table_name, id) VALUES ('review_notes', 0);''')))
print(list(col.db.execute('''INSERT INTO last_inserted_row (table_name, id) VALUES ('reviews', 0);''')))
print(list(col.db.execute('''SELECT * FROM last_inserted_row;''')))

[]
[]
[]
[['revlog', 0], ['review_notes', 0], ['reviews', 0]]


In [5]:
print(list(col.db.execute('''DELETE FROM reviews WHERE id = 9999999999999;''')))

[]


In [6]:
print(list(col.db.execute('''SELECT * FROM reviews ORDER BY id DESC limit 5;''')))

[[1673639270993, 1672682213036, 2070, 1, -600, -600, 2150, 4733, 2, '', '', 0], [1673628209710, 1672682213036, 2069, 1, -600, -600, 2150, 5645, 2, '', '', 0], [1673624998645, 1672682213036, 0, 1, -600, -600, 2150, 4912, 2, '', '', 0], [1673582757562, 1672682213036, 2067, 1, -600, -600, 2150, 4328, 2, '', '', 0], [1673577918034, 1672682213036, 2062, 1, -600, -600, 2150, 3006, 2, '', '', 0]]


In [10]:
print(list(col.db.execute('''SELECT min(r1.id + 1) FROM reviews r1 WHERE (r1.id >= 1673639270993) AND NOT EXISTS (SELECT 1 FROM reviews r2 WHERE (r2.id == r1.id + 1))''')))

[[1673639270994]]


In [27]:
print(list(col.db.execute('''SELECT r1.id + 1 FROM revlog r1 WHERE (r1.id >= 1659295892836) AND NOT EXISTS (SELECT 0 FROM revlog r2 WHERE (r2.id == r1.id + 1)) ORDER BY r1.id ASC LIMIT 5''')))

[[1659295892838], [1659295895776], [1659295898291], [1659295928603], [1659295943382]]


In [32]:
print(list(col.db.execute('''SELECT r1.id FROM revlog r1 WHERE (r1.id >= 1641539675765) AND NOT EXISTS (SELECT '' FROM revlog r2 WHERE (r2.id == r1.id + 1)) ORDER BY r1.id ASC LIMIT 5''')))

[[1641539675765], [1641539694056], [1641539712967], [1641539725186], [1641539735693]]
1673656038.020 GET /_anki/legacyPageData
1673656038.022 GET /_anki/legacyPageData
1673656038.036 GET /_anki/css/webview.css
1673656038.036 GET /_anki/css/deckbrowser.css
1673656038.038 GET /_anki/js/webview.js
1673656038.038 GET /_anki/js/vendor/jquery.min.js
1673656038.039 GET /_anki/js/vendor/jquery-ui.min.js
1673656038.039 GET /_anki/js/deckbrowser.js
1673656038.044 GET /_anki/css/webview.css
1673656038.044 GET /_anki/css/toolbar-bottom.css
1673656038.045 GET /_anki/css/toolbar.css
1673656038.045 GET /_anki/js/vendor/jquery.min.js
1673656038.046 GET /_anki/js/webview.js
1673656038.046 GET /_anki/imgs/gears.svg
1673656638.129 GET /_anki/legacyPageData
1673656638.130 GET /_anki/legacyPageData
1673656638.143 GET /_anki/css/webview.css
1673656638.146 GET /_anki/css/webview.css
1673656638.146 GET /_anki/css/toolbar.css
1673656638.147 GET /_anki/css/toolbar-bottom.css
1673656638.147 GET /_anki/js/webview

In [18]:
print(list(col.db.execute('''SELECT (SELECT coalesce(min(r1.id + 1),10) FROM reviews r1 WHERE (r1.id >= 1659295892836) AND NOT EXISTS (SELECT 1 FROM reviews r2 WHERE (r2.id == r1.id + 1)))''')))

[[1659295892838]]


In [33]:
print(list(col.db.execute('''SELECT r1.id FROM revlog AS r1 WHERE r1.id >=1641539675765 AND r1.id + 1 NOT IN (SELECT r2.id FROM revlog AS r2 WHERE r1.id >= 1641539675765 LIMIT 20) LIMIT 20''')))

[[1641539675765], [1641539694056], [1641539712967], [1641539725186], [1641539735693], [1641539756255], [1641539772321], [1641539781780], [1641539802437], [1641539812613], [1641539823996], [1641539835562], [1641539857396], [1641880534299], [1641880544534], [1641880549599], [1641880568989], [1641880574890], [1641880579317], [1641880584238]]


In [35]:
print(list(col.db.execute('''SELECT id FROM revlog where id == 1641539675766''')))

[]
1673657238.063 GET /_anki/legacyPageData
1673657238.064 GET /_anki/legacyPageData
1673657238.077 GET /_anki/css/webview.css
1673657238.079 GET /_anki/css/webview.css
1673657238.080 GET /_anki/css/toolbar.css
1673657238.081 GET /_anki/css/toolbar-bottom.css
1673657238.082 GET /_anki/js/webview.js
1673657238.082 GET /_anki/js/vendor/jquery.min.js
1673657238.083 GET /_anki/css/deckbrowser.css
1673657238.084 GET /_anki/js/vendor/jquery-ui.min.js
1673657238.085 GET /_anki/js/webview.js
1673657238.086 GET /_anki/js/deckbrowser.js
1673657238.088 GET /_anki/js/vendor/jquery.min.js
1673657238.091 GET /_anki/imgs/gears.svg
1673657838.054 GET /_anki/legacyPageData
1673657838.055 GET /_anki/legacyPageData
1673657838.066 GET /_anki/css/webview.css
1673657838.068 GET /_anki/css/webview.css
1673657838.069 GET /_anki/css/toolbar.css
1673657838.070 GET /_anki/css/deckbrowser.css
1673657838.070 GET /_anki/js/webview.js
1673657838.071 GET /_anki/js/vendor/jquery.min.js
1673657838.073 GET /_anki/js/ven

In [11]:
print(list(col.db.execute('''SELECT 1673639270994''')))

[[1673639270994]]
1673648838.006 GET /_anki/legacyPageData
1673648838.007 GET /_anki/legacyPageData
1673648838.020 GET /_anki/css/webview.css
1673648838.021 GET /_anki/css/deckbrowser.css
1673648838.022 GET /_anki/js/webview.js
1673648838.022 GET /_anki/js/vendor/jquery.min.js
1673648838.023 GET /_anki/imgs/gears.svg
1673648838.024 GET /_anki/js/vendor/jquery-ui.min.js
1673648838.024 GET /_anki/js/deckbrowser.js
1673648838.028 GET /_anki/css/webview.css
1673648838.028 GET /_anki/css/toolbar.css
1673648838.029 GET /_anki/js/webview.js
1673648838.029 GET /_anki/css/toolbar-bottom.css
1673648838.030 GET /_anki/js/vendor/jquery.min.js
1673649438.106 GET /_anki/legacyPageData
1673649438.107 GET /_anki/legacyPageData
1673649438.117 GET /_anki/css/webview.css
1673649438.119 GET /_anki/css/toolbar.css
1673649438.119 GET /_anki/css/toolbar-bottom.css
1673649438.120 GET /_anki/js/webview.js
1673649438.121 GET /_anki/js/vendor/jquery.min.js
1673649438.122 GET /_anki/css/webview.css
1673649438.125